In [ ]:
import numpy as np
freq_bands = dict(                #the frequency bands of interest for the analysis
                delta=(1, 4), 
                theta=(4, 8), 
                alpha=(8, 12), 
                beta=(13, 29), 
                gamma=(30, 80))

In [ ]:
log_freq_bands = dict()

In [ ]:
for band in freq_bands:
    lower, upper = freq_bands[band]
    band_range = upper - lower +1
    log_freq_bands[band] = (np.logspace(np.log10(lower), np.log10(upper), band_range))

In [ ]:
log_freq_bands

In [ ]:
import mne
rawfile = "/home/idrael/DATA/MEG/BIDS_clinic/derivatives/sub-BF28011991/meg/BF28011991_Block1_trans_tsss-eve-epo.fif"
epochs = mne.read_epochs(rawfile, preload= True)

In [ ]:
events = epochs.event_id
events

In [ ]:
gr_1 = epochs["Gr_1"].load_data().average

In [ ]:
for event in events.keys():
    print(event)

# Dipole visualization

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from configuration import (subjects, n_jobs, bids_root, use_source_model_for_sourceloc, inv_loose_option,
                            use_fwd_model_for_sourceloc, do_volume_source_loc, source_loc_methods,
                            pick_meg, pick_eeg, concat_raws, signal_to_noise_ratio, minimum_norm_ori,
                            peaks_tmin, peaks_tmax, peaks_mode, peaks_nr_of_points, dip_times)
import mne
from utils.utils import FileNameRetriever, RawPreprocessor, get_peak_points
import glob
from nilearn.plotting import plot_anat

mne.viz.set_3d_backend("pyvista")

fnr =FileNameRetriever(bids_root)
prepper = RawPreprocessor()

snr = signal_to_noise_ratio
lambda2 = 1. / snr ** 2

for subj in subjects:
    meg_folder = fnr.get_filename(subj, "meg")
    epos = glob.glob(meg_folder + "/*epo.fif")
    subjects_dir = fnr.get_filename(subj=subj, file="subjects_dir")
    bem_sol = fnr.get_filename(subj=subj, file=use_fwd_model_for_sourceloc)
    trans_file = fnr.get_single_trans_file(subj)
    for epo in epos:
        epochs = mne.read_epochs(epo)
        print(f"Event-IDs are: {epochs.event_id}")
        events = epochs.event_id
        print(f"\n\n\nevents: {events}")

        noise_cov = mne.compute_covariance(epochs, tmax=-1, 
                                    #projs=, 
                                    method='auto',
                                    n_jobs=n_jobs)

        data_cov = mne.compute_covariance(epochs,
                                    tmin=-0.5, 
                                    tmax=0.3, 
                                    #projs=, 
                                    method='auto',
                                    n_jobs=n_jobs)

        for event in events.keys():
            eventname = str(event)
            if eventname == "ignore_me" or eventname == "AAA":
                pass
            else:
                e = epochs[eventname].load_data().average()
                spike_folder = fnr.get_filename(subj, "spikes")
                e_folder = os.path.join(spike_folder, eventname)
                cp_folder = os.path.join(spike_folder, eventname, "custom_pics")
                cts_folder = os.path.join(spike_folder, eventname, "custom_time_series")
                gp_folder = os.path.join(spike_folder, eventname, "generic_pics")
                folders = [e_folder, cp_folder, cts_folder, gp_folder]
                if not os.path.isdir(e_folder):
                    for f in folders:
                        print(f)
                        os.mkdir(f)
                
                src_file = fnr.get_filename(subj, use_source_model_for_sourceloc)
                if os.path.isfile(src_file):
                    src = mne.read_source_spaces(src_file)
                else:
                    src = mne.setup_source_space(subj, spacing = use_source_model_for_sourceloc, 
                                                subjects_dir = subjects_dir, 
                                                n_jobs=n_jobs, 
                                                verbose=True)

                fwd = mne.make_forward_solution(e.info, src=src, bem=bem_sol,
                                            trans=trans_file, 
                                            meg=pick_meg, eeg=pick_eeg, mindist=0.2, 
                                            ignore_ref=False, 
                                            n_jobs=n_jobs, verbose=True)
                             
                inv = mne.minimum_norm.make_inverse_operator(e.info, forward=fwd, noise_cov=noise_cov, 
                                            loose=inv_loose_option, depth=0.8)
    

            

In [3]:
for m in source_loc_methods:
    stc_name = "stc_" + m + "_" + eventname
    stc_name = 'stc_' + m
    for start, stop in dip_times.values():
        dip_epoch = e.copy().crop(start, stop).pick('meg')
        ecd = mne.fit_dipole(dip_epoch, noise_cov, bem_sol, trans=trans_file)[0]
        best_idx = np.argmax(ecd.gof)
        best_time = ecd.times[best_idx]
        trans = mne.read_trans(trans_file)
        mri_pos = mne.head_to_mri(ecd.pos, mri_head_t=trans, subject=subj, subjects_dir=subjects_dir)
        t1_file_name = os.path.join(subjects_dir, subj, 'mri', 'T1.mgz')
        stoptime = str(abs(int(stop*1000)))
        if stoptime == "5":
            stoptime = "05"
        title = str(eventname + ' - ECD @ minus ' + stoptime + ' ms')

        # visualization
        
        fig, axes = plt.subplots(nrows=2, ncols=2,
                                    gridspec_kw=dict(width_ratios=[3, 1],
                                    top=0.85))
        fig.suptitle(f"{eventname} - Equivalent current dipole model - minus {stoptime} ms")
        plot_anat(t1_file_name, cut_coords=mri_pos[0], axes=axes[0])
        fig.add_subplot(2,2,1, projection="3d")
        ecd.plot_locations(trans, subj, subjects_dir, mode="orthoview")
        t1_f_name_pic = ('img_ecd_' + eventname + '_' + '_Dipol_' + stoptime + '.png')
        t1_f_name_pic = os.path.join(e_folder, t1_f_name_pic)
        fig.savefig(t1_f_name_pic)
        plt.close("all")

NameError: name 'eventname' is not defined

# Hippocampal segmentation

In [ ]:

import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np

In [ ]:
columns = ["Index", "SegId", "NVoxels", "Volume_mm3", "StructName", "normMean", "normStdDev", "normMin", "normMax", "normRange"]
subjects = ["NP21062020", "NP25062020", "NP02072020", "NP16072020"]

In [1]:
import matplotlib
matplotlib.rcParams.keys()


KeysView(RcParams({'_internal.classic_mode': False,
          'agg.path.chunksize': 0,
          'animation.avconv_args': [],
          'animation.avconv_path': 'avconv',
          'animation.bitrate': -1,
          'animation.codec': 'h264',
          'animation.convert_args': [],
          'animation.convert_path': 'convert',
          'animation.embed_limit': 20.0,
          'animation.ffmpeg_args': [],
          'animation.ffmpeg_path': 'ffmpeg',
          'animation.frame_format': 'png',
          'animation.html': 'none',
          'animation.html_args': [],
          'animation.writer': 'ffmpeg',
          'axes.autolimit_mode': 'data',
          'axes.axisbelow': 'line',
          'axes.edgecolor': 'black',
          'axes.facecolor': 'white',
          'axes.formatter.limits': [-5, 6],
          'axes.formatter.min_exponent': 0,
          'axes.formatter.offset_threshold': 4,
          'axes.formatter.use_locale': False,
          'axes.formatter.use_mathtext': False,
        